In [23]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

pd.set_option('display.max_columns', None)

In [24]:
hpms = pd.read_csv('Transportation/CO_VMT18/co_aadt_2018_updated.csv')
hpms.drop(columns='Unnamed: 0',inplace=True)
hpms['begin_point']=hpms['begin_point'].round(3)
hpms['end_point']=hpms['end_point'].round(3)
hpms['unique_route'] = hpms['route_id']+'_'+hpms['begin_point'].astype(str)+'-'+hpms['end_point'].astype(str)
hpms.set_index('unique_route',inplace=True)

In [26]:
aadt_cols = ['aadt_Motorcycle',
       'aadt_PassengerCars', 'aadt_LightTrucks', 'aadt_Buses', 'aadt_SUtrucks',
       'aadt_CUtrucks']

In [27]:
vmt_cols = ['vmt_Motorcycle',
       'vmt_PassengerCars', 'vmt_LightTrucks', 'vmt_Buses', 'vmt_SUtrucks',
       'vmt_CUtrucks']

In [28]:
co_vmt18 = pd.read_csv('Transportation/CO_VMT18/CO_VMT18.csv')

In [29]:
missing_truck_inds = co_vmt18['aadt_SUtrucks'].isna() & co_vmt18['aadt_CUtrucks'].isna()

In [30]:
for uniqueroute in co_vmt18[missing_truck_inds]['unique_route'].unique():
    hpms_uniqueroute = hpms.loc[uniqueroute]
    co_vmt18_uniqueroute_inds = co_vmt18['unique_route']==uniqueroute
    aadt_SUtrucks = hpms_uniqueroute['aadt_SU trucks']
    aadt_CUtrucks = hpms_uniqueroute['aadt_CU trucks']
    missingtruck_uniqueroute_inds = missing_truck_inds & co_vmt18_uniqueroute_inds
    segment_mi = co_vmt18.loc[missingtruck_uniqueroute_inds,'segment_mi']
    prop_area = co_vmt18.loc[missingtruck_uniqueroute_inds,'prop_area']
    co_vmt18.loc[missingtruck_uniqueroute_inds,'aadt_SUtrucks']= aadt_SUtrucks
    co_vmt18.loc[missingtruck_uniqueroute_inds,'aadt_CUtrucks']= aadt_CUtrucks
    co_vmt18.loc[missingtruck_uniqueroute_inds,'vmt_SUtrucks'] = aadt_SUtrucks * segment_mi * prop_area
    co_vmt18.loc[missingtruck_uniqueroute_inds,'vmt_CUtrucks'] = aadt_CUtrucks * segment_mi * prop_area
    

In [31]:
co_vmt18['vmt_Total'] = co_vmt18[vmt_cols].sum(axis=1)

In [32]:
co_vmt18.to_csv('Transportation/CO_VMT18_fixed.csv')